In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
df_train  = pd.read_parquet('drive/MyDrive/data/yellow_tripdata_2022-01.parquet')
df_val = pd.read_parquet('drive/MyDrive/data/yellow_tripdata_2022-02.parquet')

In [4]:
df_train['duration'] = df_train.tpep_dropoff_datetime - df_train.tpep_pickup_datetime
df_train['duration'] = df_train.duration.dt.total_seconds() / 60

In [5]:
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val['duration'] = df_val.duration.dt.total_seconds() / 60

In [6]:
df_train[(df_train.duration >= 1) & (df_train.duration <= 60)].copy().shape[0] / df_train.shape[0]

0.9827547930522406

In [7]:
df_train = df_train[(df_train.duration >= 1) & (df_train.duration <= 60)].copy()

In [8]:
from sklearn.preprocessing import OneHotEncoder

In [9]:

categorical = ['PULocationID', 'DOLocationID']

In [10]:

#del df_val

In [11]:
df_train[categorical] = df_train[categorical].astype(str)
df_val[categorical] = df_val[categorical].astype(str)

In [12]:
temp = df_train.sample(frac=0.75)

In [13]:
ohe = OneHotEncoder()
array_hot_encoded = pd.DataFrame(ohe.fit_transform(temp[categorical]).toarray())

In [14]:

array_hot_encoded.shape

(1816080, 511)

In [15]:

temp

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
1622370,1,2022-01-22 11:36:24,2022-01-22 11:41:49,1.0,0.80,1.0,N,229,141,1,5.5,2.5,0.5,0.88,0.00,0.3,9.68,2.5,0.0,5.416667
1167331,1,2022-01-16 18:51:55,2022-01-16 19:07:46,1.0,1.80,1.0,N,107,158,1,11.5,2.5,0.5,2.00,0.00,0.3,16.80,2.5,0.0,15.850000
314203,2,2022-01-05 17:58:00,2022-01-05 17:59:38,1.0,0.41,1.0,N,140,262,1,3.5,1.0,0.5,1.00,0.00,0.3,8.80,2.5,0.0,1.633333
2356759,2,2022-01-31 16:41:42,2022-01-31 16:44:16,1.0,0.60,1.0,N,43,238,2,4.0,1.0,0.5,0.00,0.00,0.3,8.30,2.5,0.0,2.566667
1047164,2,2022-01-15 10:17:30,2022-01-15 10:44:28,1.0,11.04,1.0,N,100,14,1,33.5,0.0,0.5,8.67,6.55,0.3,52.02,2.5,0.0,26.966667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296809,1,2022-01-18 16:40:10,2022-01-18 16:46:28,1.0,0.90,1.0,N,262,141,2,6.0,3.5,0.5,0.00,0.00,0.3,10.30,2.5,0.0,6.300000
1558281,2,2022-01-21 16:24:50,2022-01-21 16:44:46,1.0,3.08,1.0,N,161,211,1,14.0,1.0,0.5,4.58,0.00,0.3,22.88,2.5,0.0,19.933333
2261662,2,2022-01-30 12:44:33,2022-01-30 12:54:55,1.0,1.57,1.0,N,87,148,1,9.0,0.0,0.5,2.46,0.00,0.3,14.76,2.5,0.0,10.366667
1527438,2,2022-01-21 11:18:03,2022-01-21 11:27:06,1.0,0.74,1.0,N,79,232,1,7.0,0.0,0.5,2.06,0.00,0.3,12.36,2.5,0.0,9.050000


In [16]:
#train_dicts = array_hot_encoded.to_dict(orient='records')


In [17]:
y_train = temp['duration']
lr = LinearRegression()
lr.fit(array_hot_encoded, y_train)

LinearRegression()

In [18]:
y_pred = lr.predict(array_hot_encoded)
mean_squared_error(y_train, y_pred, squared=False)

6.988911809913138

In [19]:

array_hot_encoded_val = pd.DataFrame(ohe.transform(df_val[categorical]).toarray())

In [ ]:

y_val = df_val['duration']

In [19]:
y_pred_val = lr.predict(array_hot_encoded_val = pd.DataFrame(ohe.transform(df_val[categorical]).toarray()))
mean_squared_error(y_pred_val, y_val, squared=False)